In [1]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import json
import os
import cv2
import torch
import numpy as np
import random

In [43]:
class MyDataset(Dataset):
    def __init__(self, j):
        # with open('annotations.json', 'r', encoding='utf-8') as file:
        #     data = json.load(file)
        with open(j, 'r') as json_file:
            json_data = json.load(json_file)
        # name フィールドの値をリストに抽出
        name_list = [item['name'] for item in json_data if 'name' in item]

        self.pitch_list = []   
        for item in json_data:
            for target in name_list:
                pitch_time = []
                if item["name"] == target:
                    for attribute in item["attributes"]:
                        if attribute["key"] == "pitch":
                            pitch_time.extend(attribute["value"])
                for i in range (len(pitch_time)):
                    self.pitch_list.append([target, pitch_time[i]])
        self.clip_length= 128


    def __len__(self):
        return len(self.pitch_list)
    
    def __getitem__(self, index):
        data_transform = transforms.Compose([transforms.ToTensor(),
                                            ])
        print(self.pitch_list[index])
        name, [pitch_start, pitch_end] = self.pitch_list[index]
        random_number = random.randint(0, self.clip_length // 2)
        clip_start = max(0,pitch_start - random_number)
        clip_end = clip_start + self.clip_length
        img_list = []
        anno_list = []
        for i in range(clip_start, clip_end):
            if pitch_start <= i <= pitch_end:
                anno_list.append(1)
            else:
                anno_list.append(0)
            #print(clip_start, pitch_start, random_number)
            #print(f'frames/frame_{str(i).zfill(4)}.jpg')
            # clip_frame = cv2.imread(f'frames/frame_{str(i).zfill(4)}.jpg')
            # img_tensor = data_transform(clip_frame)
            # img_list.append(img_tensor)
        return img_list, anno_list                    








In [44]:
my_dataset = MyDataset("/Users/hibiyagenki/Downloads/Implement/annotations.json")
img, anno = my_dataset[8]
print(anno)


['4b_vsWASEDA.mp4', [6440, 6539]]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


プロジェクトフォルダを作る
固定長のフレーム数（ただしピッチがある程度含まれるようにする）のフレームのインデックスを生成
2.のフレームを取得
2.の対応するアノテーションを取得
datasetの引数に，固定調のフレーム数とビデオの親ディレクトリのパスとアノテーションファイルのパスを指定
ピッチが部分的に含まれているもの×ピッチ数×動画数
10-50でランダム
annotation.jsonを読み込んだ際に動画のnameだけのリストも作成
固定長は128か256